In [ ]:
from u_cmab import Fourier
from u_cmab import Static_UM
from u_cmab import plot_results
import simulation_code

import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

# Parameters
- `window`: size of the windowed average uplift regret
- `static_dataset_size`: amount of random experiments conducted and provided to the static uplift model
- `total_experiment_count`: length of the total run
- `run_count`: total amount of simulations created, take note before increasing as this will increase runtime significantly

In [ ]:
window = 150
static_dataset_size = 2500#5000
total_experiment_count = 15000
run_count = 10# ** 2

In [ ]:
fourier_ids = np.empty([run_count,total_experiment_count])
fourier_reg_ids = np.empty([run_count, total_experiment_count])
static_ids = np.empty([run_count, total_experiment_count])
static_params = np.empty([run_count, 2])

for i in range(run_count):    
    ply_q = np.random.randint(6,10)
    tau = (.45 - .25) * np.random.ranf() + .25
        #  the order of the Fourier approx. (O) is best chosen in 
        #  function of D as O and D affect the amount of frequency 
        #  vectors exponentially, causing a curse of dimensionality
        #  [Konindaris et al., 2011].
    D = np.random.randint(2,4)   
    O = 7-D 
            

    K=np.random.normal(1,.5, ply_q+1)
    sim = simulation_code.Simulation(
        D=D, base_functions=np.array([
            simulation_code.Sine_Base(f=np.random.randint(6,10)), 
            simulation_code.Sine_Base(f=np.random.randint(4,7))
        ]))
    
    fourier = Fourier(sim, order=O)
    fourier_id, fourier_dm, fourier_reg_id, tot_rew, exec_ac, tot_rew_reg, exec_ac_reg = fourier.run(
        epsilon=.07, 
        alpha=(O-1)/1000,
        tau=tau, window=window, 
        lifetime=total_experiment_count)
    print("--Fourier {} Completed--".format(i+1))
    
    static = Static_UM(RP_run_history=fourier.RP_run_history)
    up, static_id = static.run(
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window)
    
    fourier_ids[i] = fourier_id
    fourier_reg_ids[i] = fourier_reg_id
    static_ids[i] = static_id
    static_params[i] = list(up.rand_search_.best_params_.values())

In [ ]:
avg_ids = [np.mean(fourier_ids, axis=0), np.mean(static_ids, axis=0), np.mean(fourier_reg_ids, axis=0)]
stds = [np.std(fourier_ids, axis=0), np.std(static_ids, axis=0), np.std(fourier_reg_ids, axis=0)]

In [ ]:
line_args=[
    {"color": "dodgerblue", "linewidth": 2, "zorder": 103, "label": "U-CMAB"}, 
    {"color": "firebrick", "linewidth": 2, "zorder": 102, "label": "Random Forest (ADWIN)"}, 
    {"color": "forestgreen", "linewidth": 2, "zorder": 103, "label": "CMAB"}]
band_args=[
    {"color": "turquoise", "alpha": .35, "linewidth":0, "zorder": 101}, 
    {"color": "deeppink", "alpha": .15, "linewidth":0, "zorder": 100}, 
    {"color": "lime", "alpha": .25, "linewidth":0, "zorder": 100}]
all_lines_args=[
    {"color": "deepskyblue", "alpha": .1, "linewidth": .5}, 
    {"color": "crimson", "alpha": .1, "linewidth": .5}, 
    {"color": "black", "alpha": .1, "linewidth": .5}]
std_args=[
    {"color": "deepskyblue", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]}, 
    {"color": "crimson", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]}, 
    {"color": "black", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]}]

stop_RP_args={"color": "tab:gray", "linewidth":2, "linestyle": "--", "dashes": [5,5], "alpha": .8}
stop_RP_text_args={"y": .95, "color": "tab:gray", "size": 19, "fontstyle": "italic"}
drift_line_args={"linestyle": ":", "color": "tab:green", "alpha": 1, "linewidth": 3}

fig, ax = plot_results(avg_ids, stds, np.array([]), 
                 line_args=line_args, band_args=band_args, all_lines_args=all_lines_args, std_args=std_args,
                stop_RP=static_dataset_size, stop_RP_args=stop_RP_args, stop_RP_text_args=stop_RP_text_args)